In [0]:
import pandas as pd
import pickle
import random

# ---------- Load Saved Model and Transformers ----------
model_path = "/Workspace/Users/m.a.arya@accenture.com/randomforest.pkl"
scaler_path = "/Workspace/Users/m.a.arya@accenture.com/scaler.pkl"
encoder_path = "/Workspace/Users/m.a.arya@accenture.com/label_encoders.pkl"

with open(model_path, 'rb') as f:
    model = pickle.load(f)
with open(scaler_path, 'rb') as f:
    scaler = pickle.load(f)
with open(encoder_path, 'rb') as f:
    label_encoders = pickle.load(f)


# ---------- Input Widgets ----------
dbutils.widgets.text("amount", "1000", "Amount")
dbutils.widgets.dropdown("hour", "12", [str(i) for i in range(24)], "Hour of Day")
dbutils.widgets.dropdown("day_of_week", "Monday", ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], "Day of Week")
dbutils.widgets.dropdown("upi_channel", "PhonePe", ["GPay", "BHIM", "PayTM", "PhonePe"], "UPI Channel")
dbutils.widgets.dropdown("merchant_category", "Utilities",['Utilities', 'Shopping', 'Food', 'Travel', 'Education', 'Healthcare'], "Merchant Category")
dbutils.widgets.dropdown("device_os", "Android", ["Android", "iOS", "Windows", "Other"], "Device OS")
dbutils.widgets.text("transaction_city", "Mumbai", "Transaction City")
dbutils.widgets.text("transaction_state", "Maharashtra", "Transaction State")
dbutils.widgets.dropdown("transaction_status", "Completed", ["Completed", "Pending", "Failed"], "Transaction Status")

# ---------- Feature Construction ----------
input_data = {
    "amount": float(dbutils.widgets.get("amount")),
    "hour": int(dbutils.widgets.get("hour")),
    "day_of_week": dbutils.widgets.get("day_of_week"),
    "transaction_amount_deviation": round(random.uniform(-100, 100), 2),  # Simulate deviation
    "is_high_value": 1 if float(dbutils.widgets.get("amount")) > 3000 else 0,
    "is_odd_hour": 1 if int(dbutils.widgets.get("hour")) < 6 or int(dbutils.widgets.get("hour")) > 22 else 0,
    "upi_channel": dbutils.widgets.get("upi_channel"),
    "merchant_category": dbutils.widgets.get("merchant_category"),
    "device_os": dbutils.widgets.get("device_os"),
    "transaction_city": dbutils.widgets.get("transaction_city"),
    "transaction_state": dbutils.widgets.get("transaction_state"),
    "transaction_status": dbutils.widgets.get("transaction_status")
}

df_input = pd.DataFrame([input_data])

# ---------- Preprocess ----------
categorical_cols = [
    'day_of_week',
    'upi_channel',
    'transaction_status',
    'merchant_category',
    'device_os',
    'transaction_city',
    'transaction_state'
]

numeric_cols = [
    'amount',
    'hour',
    'transaction_amount_deviation',
    'is_high_value',
    'is_odd_hour'
]

# Encode
for col in categorical_cols:
    encoder = label_encoders[col]
    df_input[col + "_enc"] = encoder.transform(df_input[col].astype(str))

# Scale
df_input[numeric_cols] = scaler.transform(df_input[numeric_cols])

# Feature columns
feature_cols = [col + "_enc" for col in categorical_cols] + numeric_cols
X_input = df_input[feature_cols]

# ---------- Predict ----------
y_pred = model.predict(X_input)
y_proba = model.predict_proba(X_input)[:, 1]

# ---------- Display Result in Widget ----------
df_input["fraud_prediction"] = y_pred
df_input["fraud_probability"] = y_proba

prediction = "Fraudulent" if y_pred[0] == 1 else "Legitimate"
color = "#ff4d4d" if y_pred[0] == 1 else "#5cd65c"
confidence = round(y_proba[0] * 100, 2)

html = f"""
<div style="padding:15px;border-radius:10px;background-color:{color};color:white;width:300px;">
    <h3>Prediction: {prediction}</h3>
    <p>Confidence: {confidence}%</p>
</div>
"""

displayHTML(html)


Prediction: Fraudulent 
 Confidence: 51.0%